In [8]:
import pandas as pd

annual_report_info=pd.read_csv('../data/train/annual_report_info.csv')#企业的年报基本信息

In [9]:
annual_report_info.isnull().sum()

id                   0
ANCHEYEAR            0
STATE                5
FUNDAM           16848
MEMNUM           22521
FARNUM           22521
ANNNEWMEMNUM     22521
ANNREDMEMNUM     22521
EMPNUM              15
EMPNUMSIGN        5717
BUSSTNAME         4870
COLGRANUM         2509
RETSOLNUM         2509
DISPERNUM         2509
UNENUM            2509
COLEMPLNUM        2509
RETEMPLNUM        2509
DISEMPLNUM        2509
UNEEMPLNUM        2509
WEBSITSIGN          33
FORINVESTSIGN     6061
STOCKTRANSIGN     9043
PUBSTATE            20
dtype: int64

In [10]:
#处理base_info的数据
#空值大于0.5的列都删除掉
annual_report_info_clean=annual_report_info.dropna(thresh=annual_report_info.shape[0]*0.5,how='all',axis=1)

In [11]:
annual_report_info_clean.isnull().sum()

id                  0
ANCHEYEAR           0
STATE               5
EMPNUM             15
EMPNUMSIGN       5717
BUSSTNAME        4870
COLGRANUM        2509
RETSOLNUM        2509
DISPERNUM        2509
UNENUM           2509
COLEMPLNUM       2509
RETEMPLNUM       2509
DISEMPLNUM       2509
UNEEMPLNUM       2509
WEBSITSIGN         33
FORINVESTSIGN    6061
STOCKTRANSIGN    9043
PUBSTATE           20
dtype: int64

In [12]:
#对object类型进行编码
annual_report_info_clean['BUSSTNAME']=annual_report_info_clean['BUSSTNAME'].fillna("无")
dic = {'无':-1,'开业':0, '歇业':1, '停业':2, '清算':3}
buf = pd.DataFrame()
buf_group = annual_report_info_clean.groupby('BUSSTNAME',sort=False)
for name,group in buf_group:
    group['BUSSTNAME'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
buf=buf.fillna(-1)
#
buf_group = buf.groupby('id',sort=False).agg('mean')
buf=pd.DataFrame(buf_group).reset_index()
annual_report_info_clean=buf.drop(['ANCHEYEAR'],axis=1)

In [13]:
annual_report_info_clean.to_csv('annual_report_info_feature.csv',index = False)

In [14]:
annual_report_info_clean.describe()

,STATE,EMPNUM,EMPNUMSIGN,BUSSTNAME,COLGRANUM,RETSOLNUM,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
count,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000,8937.000000
mean,1.998433,9.060879,0.981162,-0.195921,0.616838,-0.048930,-0.077683,0.157503,1.126765,-0.021206,-0.063599,0.461257,1.965844,1.019251,0.663133,2.724743
std,0.036739,101.814634,1.329025,0.536274,2.965080,0.596557,0.304094,1.805977,6.321264,0.766395,0.411494,3.970381,0.175788,1.379495,1.468833,0.457251
min,0.500000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,2.000000,1.500000,-1.000000,-0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,-1.000000,-1.000000,2.500000
50%,2.000000,2.500000,1.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,1.750000,3.000000
75%,2.000000,5.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,3.000000
max,2.000000,8050.750000,2.000000,3.000000,177.000000,40.000000,2.500000,102.500000,309.500000,29.250000,10.500000,132.250000,2.000000,2.000000,2.000000,3.000000
